In [20]:
#加载需要的包
import re

import pandas as pd
import json
import ast
import os

机构组织生成 | institutions attribute generation

根据爬取处理好的数据从百度百科中获取机构的属性

In [3]:
#获取机构列表
entity = pd.read_csv('../data/new_data/neo4j_file_1/entity.csv')
institutions = entity[entity[':LABEL']=='机构']['name'].array.to_numpy().tolist()

#构建list并写入
with open('../data/second/crawler/download_url.txt','w',encoding='utf-8') as outfile:
    for i in institutions:
        outfile.write('https://baike.baidu.com/item/'+i+'\n')

In [4]:
#从爬取到的文件中，获得数据并进行处理
with open('../data/second/crawler/crawler.json','r',encoding='utf-8') as infile:
    crawlers = json.load(infile)
result = {}
for crawler in crawlers:
    for institution in crawler:
        temp = {}
        institution_name = crawler['文本框值']
        alias = crawler['字段2']
        alias = alias.strip()
        abstract = crawler['字段3'].strip().replace(' ','').replace('\n','')
        if alias: temp['别名'] = alias
        re1 = re.findall(r".*\[(.*)\].*",abstract)
        for i in re1:
            abstract = re.sub("\["+i+"\]",'',abstract,count=0)
        temp['摘要'] = abstract

        describe = crawler['字段5'].strip().replace('\xa0','').replace(' ','')
        re2 = re.findall(r".*\[(.*)\].*",describe)
        for i in re2:
            describe = re.sub("\["+i+"\]",'',describe,count=0)
        describe = describe.split('\n')
        name = ''
        for i in describe:
            if i != '':
                if name == '':
                    name = i
                else:
                    temp[name] = i
                    name = ''
        result[institution_name] = temp

with open('../data/second/crawler/crawler_result.json','w',encoding='utf-8') as outfile:
    json.dump(result,outfile,indent=2,ensure_ascii=False)
result

{'商务部': {'别名': '中华人民共和国商务部',
  '摘要': '中华人民共和国商务部是主管我国国内外贸易和国际经济合作的国务院组成部门，承担制订和实施我国国内外经济贸易政策、推进扩大对外开放的重要职责。',
  '中文名': '中华人民共和国商务部',
  '外文名': "Ministry of Commerce of the People's Republic",
  '办公地址': '北京东长安街2号',
  '性质': '国务院组成部门',
  '隶属于': '中华人民共和国国务院',
  '部长': '王文涛'},
 '北京市交通委员会': {'摘要': '根据中共中央、国务院批准的北京市人民政府机构改革方案和《北京市人民政府关于机构设置的通知》（京政发〔2009〕2号），设立北京市交通委员会（简称市交通委）。',
  '中文名': '北京市交通委员会',
  '组织性质': '政府机构',
  '地址': '北京市六里桥南里甲9号首发大厦B',
  '简称': '北京市交通委'},
 '北京市发展和改革委员会': {'别名': '北京市发展和改革委员会',
  '摘要': '北京市发展和改革委员会主要负责北京市国民经济和社会发展统筹协调、经济体制改革综合协调的市政府组成部门。',
  '中文名': '北京市发展和改革委员会',
  '外文名': 'Beijing Municipal Commission of Development and Reform',
  '办公地址': '北京市通州区运河东大街55号院',
  '简称': '北京市发改委',
  '机关行政编制': '265名',
  '所属单位': '北京市人民政府',
  '内设机构': '26个',
  '机构设置': '办公室，研究室',
  '邮政编码': '100031',
  '主任': '穆鹏'},
 '北京市通信管理局': {'摘要': '北京市通信管理局是北京市行政区域内通信行业的主管部门，实行工业和信息化部与北京市政府双重领导，以工业和信息化部为主的管理体制。',
  '中文名': '北京市通信管理局',
  '所属地区': '北京市',
  '管理范围': '通信管理',
  '属性': '政府机关'},
 '北京市人力资源和社会保障局': {'别名': '北京

In [5]:
pattern = [['机构隶属','隶属机构','主管单位','隶属单位','所属单位','直属事业单位','所属机构','登记机关','隶属机关','所属部门','隶属','隶属于','机关隶属','主管部门'],
           ['主要职责','职责','管理范围'],['所获荣誉','荣誉称号'],['名称','别名','中文名','外文名','公司名称','英文名','原名','简称','英文缩写'],
           ['内设机构','机构设置'],['地址','机关驻址','地区','所属地区','办公地址','办公地点','总部地点','地点','区位'],['简介','摘要'],
           ['负责人','秘书长','领导人','局长','主任','现任行长','现任关长','院长','现任市长','主席','现任区长','部长','现任领导','现任署长','副局长','副巡视员'],
           ['组织性质','机构性质','属性','单位性质','性质','类别','机构属性'],['机构人数','行政编制','机关行政编制','员工数'],
           ['成立日期','组建时间','成立时间','组建日期','设立时间','发照时间','始建于','成立']]
import json
from collections import defaultdict
with open('../data/dynamic/crawler/crawler_result.json','r',encoding='utf-8') as infile:
    result = json.load(infile)
new_result = {}
for institution_name in result:
    temp = defaultdict(set)
    for i in result[institution_name]:
        for j in pattern:
            if i in j:
                temp[j[0]].add(result[institution_name][i])
                break
        else:
            temp[i].add(result[institution_name][i])
    new_result[institution_name] = temp

# 获得属性及数量
newcategorylist, categorylist = set(), set()
for institution_name in result:
    for i in result[institution_name]:
        categorylist.add(i)
print('*'*50,'before:','*'*50,sep='')
print(categorylist,len(categorylist))
print('*'*50,'after:','*'*50,sep='')
for institution_name in new_result:
    for i in new_result[institution_name]:
        newcategorylist.add(i)
print(newcategorylist,len(newcategorylist))

new_result1 = {}
for institution_name in new_result:
    temp = {}
    for i in new_result[institution_name]:
        temp[i] = list(new_result[institution_name][i])
    new_result1[institution_name] = temp
with open('../data/dynamic/new_triple/institution_att.json','w',encoding='utf-8') as outfile:
    json.dump(new_result1,outfile,ensure_ascii=False,indent=2)

**************************************************before:**************************************************
{'直属事业单位', '公司类型', '机构属性', '主要职责', '副局长', '负责人', '行政级别', '区位', '外文名', '属性', '垂直管理', '英文缩写', '始建于', '主席', '单位级别', '地址', '公司口号', '员工数', '性质', '简称', '分中心', '机关驻址', '组建时间', '机构性质', '副巡视员', '地区', '秘书长', '职能', '职责', '主任', '摘要', '管理单位', '现任领导', '所属机构', '经营范围', '成立日期', '现任区长', '现任市长', '所获荣誉', '机构隶属', '联系电话', '所属部门', '所属地区', '办公机构', '隶属机关', '邮政编码', '现任关长', '主管单位', '隶属机构', '领域', '成立', '设立时间', '现任署长', '组建日期', '行政职能', '管理范围', '隶属', '隶属单位', '源于', '办公地点', '机关行政编制', '现任行长', '所属单位', '别名', '机关机构', '处室', '机关隶属', '组织性质', '局长', '院长', '主管部门', '发照时间', '机构设置', '英文名', '隶属于', '成立时间', '机关属性', '部长', '原名', '单位性质', '总部', '地点', '行政编制', '公司名称', '总部地点', '官网', '类别', '中文名', '内设机构', '登记机关', '荣誉称号', '办公地址'} 92
**************************************************after:**************************************************
{'邮政编码', '公司类型', '职能', '领域', '机关属性', '主要职责', '管理单位', '负责人', '行政级别', '行政职能', '总部', '官网', '垂直管理', '源于',

In [30]:
#将三元组合并
outputpath = '../data/second/new_triple/part/institution_att.txt'
with open('../data/second/new_triple/institution_att.json','r',encoding='utf-8') as outfile:
    result = json.load(outfile)

with open(outputpath,'w',encoding='utf-8') as outfile:
    for name in result:
        for att in result[name]:
            for i in result[name][att]:
                if name and i:
                    outfile.write(str(name)+"\t"+str(att)+"\t"+str(i)+"\n")

In [31]:
#将所有的part全部转化成用\t分割的方式
triple_list = ['describe.txt','release.txt','result.txt','synergy.txt']
path = '../data/new_data/real_triple/'
outpath = '../data/second/new_triple/part/'
for file in triple_list:
    with open(outpath+file,'w',encoding='utf-8') as outfile:
        with open(path+file,'r',encoding='utf-8') as infile:
            for line in infile.readlines():
                temp1 = ast.literal_eval(line)
                outfile.write('\t'.join(temp1)+'\n')

In [32]:
#汇总
hzpath = '../data/second/new_triple/part/'
all_path = '../data/second/new_triple/'
filelist = os.listdir(hzpath)
with open(all_path+'all.txt','w',encoding='utf-8') as outfile:
    for file in filelist:
        with open(hzpath+file,'r',encoding='utf-8') as infile:
            for line in infile.readlines():
                outfile.write(line)